In [1]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import torch
import torch.autograd as autograd
import torch.optim as optim
from torch.distributions import constraints, transform_to

import pyro
import pyro.contrib.gp as gp

assert pyro.__version__.startswith('1.7.0')
pyro.set_rng_seed(1)
import numpy as np 

In [2]:
def update_posterior(x_new):
    x_new = x_new.reshape(1, -1)
    y = f(x_new) # evaluate f at new point.
    X = torch.cat([gpmodel.X, x_new]) # incorporate new evaluation
    y = torch.cat([gpmodel.y, y])
    gpmodel.set_data(X, y)
    # optimize the GP hyperparameters using Adam with lr=0.001
    optimizer = torch.optim.Adam(gpmodel.parameters(), lr=0.001)
    gp.util.train(gpmodel, optimizer)

In [3]:
def lower_confidence_bound(x, kappa=2):
    mu, variance = gpmodel(x, full_cov=False, noiseless=False)
    sigma = variance.sqrt()
    return mu - kappa * sigma

In [4]:
def find_a_candidate(x_init, lower_bound=-10, upper_bound=10):
    # transform x to an unconstrained domain
    _, numEle = x_init.shape
    #print(x_init.shape)
    if not isinstance(lower_bound, (list, tuple, torch.Tensor)):
        lower_bound = torch.tensor([lower_bound]*numEle).float()
        upper_bound = torch.tensor([upper_bound]*numEle).float()
    elif isinstance(lower_bound, torch.Tensor):
        lower_bound = lower_bound.float()
        upper_bound = upper_bound.float()
    else:
        lower_bound = torch.tensor(lower_bound).float()
        upper_bound = torch.tensor(upper_bound).float()
        
    constraint = constraints.interval(lower_bound, upper_bound)
    unconstrained_x_init = transform_to(constraint).inv(x_init)
    unconstrained_x = unconstrained_x_init.clone().detach().requires_grad_(True)
    minimizer = optim.LBFGS([unconstrained_x], line_search_fn='strong_wolfe')

    def closure():
        minimizer.zero_grad()
        x = transform_to(constraint)(unconstrained_x)
        y = lower_confidence_bound(x)
        autograd.backward(unconstrained_x, autograd.grad(y, unconstrained_x))
        return y

    minimizer.step(closure)
    # after finding a candidate in the unconstrained domain,
    # convert it back to original domain.
    x = transform_to(constraint)(unconstrained_x)
    return x.detach()

In [5]:
def next_x(lower_bound=-10, upper_bound=10, num_candidates=5):
    candidates = []
    values = []
    
        

    x_init = gpmodel.X[-1, :].reshape(1, -1)
    _, numEle = x_init.shape
    
    if not isinstance(lower_bound, (list, tuple, torch.Tensor)):
        lower_bound = torch.tensor([lower_bound]*numEle).float()
        upper_bound = torch.tensor([upper_bound]*numEle).float()
    else:
        lower_bound = torch.tensor(lower_bound).float()
        upper_bound = torch.tensor(upper_bound).float()
    for i in range(num_candidates):
        x = find_a_candidate(x_init, lower_bound, upper_bound)
        y = lower_confidence_bound(x)
        candidates.append(x)
        values.append(y)
        x_init = [(torch.rand(1)*(up - low)+low).item() for low, up in zip(lower_bound, upper_bound)]
        x_init = torch.tensor(x_init).reshape(1, -1).float()

    argmin = torch.min(torch.cat(values), dim=0)[1].item()
    return candidates[argmin]

In [7]:
def f(x):
    beta = torch.tensor([1.0, -2, -1, -2])
    xNew = torch.matmul(x, beta)
    #return torch.exp(xNew)
    return (x[:, 0]-1)**2 + (x[:, 1]-2)**2 + x[:, 2]**2 + x[:, 3]**2

In [8]:
X = torch.tensor([
    [0.0, 0.33, 0.66, 1.0],
    [-2.0, -0.33, -1.66, 0.8],
    [1.0, 2, 0, 0]]
                  )
y = f(X)

In [16]:
f(X[0, :].reshape(1, -1))

tensor([5.2245])

In [9]:
gpmodel = gp.models.GPRegression(X, y, gp.kernels.Matern52(input_dim=4),
                                 noise=torch.tensor(1), jitter=1.0e-2)

In [10]:
optimizer = torch.optim.Adam(gpmodel.parameters(), lr=0.001)
gp.util.train(gpmodel, optimizer)
for i in range(100):
    xmin = next_x(-2, 2, 10)
    print(xmin, f(xmin), lower_confidence_bound(xmin))
    update_posterior(xmin)

tensor([[ 2.0000,  2.0000,  2.0000, -2.0000]]) tensor([9.0000]) tensor([-4.0019], grad_fn=<SubBackward0>)
tensor([[ 2.0000, -2.0000,  2.0000,  2.0000]]) tensor([25.0000]) tensor([-4.9936], grad_fn=<SubBackward0>)
tensor([[ 2.0000,  2.0000, -2.0000, -2.0000]]) tensor([9.0000]) tensor([-4.5143], grad_fn=<SubBackward0>)
tensor([[-2.0000,  2.0000, -2.0000, -2.0000]]) tensor([17.0000]) tensor([-4.4977], grad_fn=<SubBackward0>)
tensor([[ 1.9997,  2.0000,  2.0000, -2.0000]]) tensor([8.9993]) tensor([-6.1539], grad_fn=<SubBackward0>)
tensor([[ 2.0000,  2.0000,  2.0000, -2.0000]]) tensor([9.0000]) tensor([-6.4613], grad_fn=<SubBackward0>)
tensor([[ 2.0000,  2.0000,  1.9999, -2.0000]]) tensor([8.9996]) tensor([-5.1315], grad_fn=<SubBackward0>)
tensor([[ 2.0000,  2.0000,  2.0000, -2.0000]]) tensor([8.9999]) tensor([-4.3158], grad_fn=<SubBackward0>)
tensor([[ 2.0000,  2.0000,  2.0000, -2.0000]]) tensor([9.0000]) tensor([-3.7431], grad_fn=<SubBackward0>)
tensor([[ 2.0000,  2.0000, -2.0000, -2.0000]

RuntimeError: torch.linalg.cholesky: U(39,39) is zero, singular U.
     Trace Shapes:
      Param Sites:
kernel.lengthscale
   kernel.variance
             noise
     Sample Sites:

In [17]:
lower_confidence_bound(X)

tensor([-122.9977, 7953.4365, -143.2686], grad_fn=<SubBackward0>)

In [18]:
f(X)

tensor([6.9428e+00, 8.1119e+03, 1.0000e+00])